<a href="https://colab.research.google.com/github/Chaitanya253/Human-activity-recognition-using-Bi-Directional-RNN/blob/main/DNA%20PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install mediapipe opencv-python numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 9.2 MB/s eta 0:00:00

In [ ]:
from google.colab import files
import cv2
import mediapipe as mp
import numpy as np
import math
import io
from PIL import Image
from google.colab.patches import cv2_imshow
import csv

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

def angle_between(a, b, c):
    ab = np.array(a) - np.array(b)
    cb = np.array(c) - np.array(b)
    cosang = np.dot(ab, cb) / (np.linalg.norm(ab) * np.linalg.norm(cb) + 1e-8)
    cosang = np.clip(cosang, -1.0, 1.0)
    return math.degrees(math.acos(cosang))

def torso_tilt_deg(shoulder_l, shoulder_r, hip_l, hip_r):
    mid_sh = ((shoulder_l[0]+shoulder_r[0])/2, (shoulder_l[1]+shoulder_r[1])/2)
    mid_hip = ((hip_l[0]+hip_r[0])/2, (hip_l[1]+hip_r[1])/2)
    vec = np.array(mid_sh) - np.array(mid_hip)
    vertical = np.array([0.0, -1.0])
    cos = np.dot(vec, vertical) / (np.linalg.norm(vec)*np.linalg.norm(vertical) + 1e-8)
    cos = np.clip(cos, -1.0, 1.0)
    return abs(math.degrees(math.acos(cos)))

def classify_activity(image_np, filename, debug=False):
    h, w = image_np.shape[:2]
    img_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)
    if not results.pose_landmarks:
        return filename, "No person detected"

    lm = results.pose_landmarks.landmark
    def xy(i): return (lm[i].x * w, lm[i].y * h)
    def vis(i): return getattr(lm[i], "visibility", 1.0)

    # Landmark indices (MediaPipe Pose)
    NOSE = 0
    MOUTH_LEFT, MOUTH_RIGHT = 9, 10
    LEFT_SHOULDER, RIGHT_SHOULDER = 11, 12
    LEFT_HIP, RIGHT_HIP = 23, 24
    LEFT_KNEE, RIGHT_KNEE = 25, 26
    LEFT_ANKLE, RIGHT_ANKLE = 27, 28
    LEFT_WRIST, RIGHT_WRIST = 15, 16

    # Coordinates
    nose = xy(NOSE)
    mouth_l, mouth_r = xy(MOUTH_LEFT), xy(MOUTH_RIGHT)
    ls, rs = xy(LEFT_SHOULDER), xy(RIGHT_SHOULDER)
    lh, rh = xy(LEFT_HIP), xy(RIGHT_HIP)
    lk, rk = xy(LEFT_KNEE), xy(RIGHT_KNEE)
    la, ra = xy(LEFT_ANKLE), xy(RIGHT_ANKLE)
    lw, rw = xy(LEFT_WRIST), xy(RIGHT_WRIST)

    # Basic metrics
    hip_width = np.linalg.norm(np.array(lh) - np.array(rh)) + 1e-8
    shoulder_width = np.linalg.norm(np.array(ls) - np.array(rs)) + 1e-8
    scale = max(shoulder_width, hip_width)
    ankle_distance = np.linalg.norm(np.array(la) - np.array(ra))
    stride_ratio = ankle_distance / scale
    torso_angle = torso_tilt_deg(ls, rs, lh, rh)
    left_knee_angle = angle_between(lh, lk, la)
    right_knee_angle = angle_between(rh, rk, ra)
    avg_knee_angle = (left_knee_angle + right_knee_angle) / 2
    hand_distance = np.linalg.norm(np.array(lw) - np.array(rw))

    # Eating heuristics (kept from previous improvements)
    mouth_center = ((mouth_l[0] + mouth_r[0]) / 2, (mouth_l[1] + mouth_r[1]) / 2)
    left_mouth = np.linalg.norm(np.array(lw) - np.array(mouth_center))
    right_mouth = np.linalg.norm(np.array(rw) - np.array(mouth_center))
    left_nose = np.linalg.norm(np.array(lw) - np.array(nose))
    right_nose = np.linalg.norm(np.array(rw) - np.array(nose))

    dist_thresh = 1.2 * scale
    y_thresh = 0.6 * h
    left_cond = (left_mouth < dist_thresh or left_nose < dist_thresh) and (lw[1] < lh[1] + y_thresh)
    right_cond = (right_mouth < dist_thresh or right_nose < dist_thresh) and (rw[1] < rh[1] + y_thresh)
    eating_condition = (left_cond or right_cond) and torso_angle < 35 and avg_knee_angle > 120

    # Hips / shoulders y for vertical relation
    hips_y = (lh[1] + rh[1]) / 2
    shoulders_y = (ls[1] + rs[1]) / 2

    # --- Enhanced sitting logic ---
    # If knees are clearly bent and hips below shoulders -> sitting (strong cue)
    bent_knees = avg_knee_angle < 140
    hips_below_shoulders = hips_y > shoulders_y + 5  # +5 px tolerance
    # Also accept deep bend (very small knee angle) as sitting even if hips relation not perfect
    deep_bend = avg_knee_angle < 120

    # Two ways to be considered sitting:
    # A) Bent knees + hips below shoulders (strong)
    # B) Deep knee bend + relatively low stride (cross-legged or tight sitting)
    sitting_condition = (bent_knees and hips_below_shoulders) or (deep_bend and stride_ratio < 0.6)

    # --- Tighter running logic (so bent-knee sit doesn't get classified as running) ---
    # Require larger stride and torso slightly forward / motion-like pose
    running_condition = (stride_ratio > 0.55) and (avg_knee_angle < 160) and (torso_angle < 35) and (hips_y <= shoulders_y + 20)

    # --- Other labels ---
    standing_condition = (torso_angle < 10 and avg_knee_angle > 160)
    sleeping_condition = (torso_angle > 60)
    playing_condition = (hand_distance > hip_width * 1.5)

    # Decide label (priority: eating -> sitting -> running -> standing -> ...)
    if eating_condition:
        label = "eating"
    elif sitting_condition:
        label = "sitting"
    elif running_condition:
        label = "running"
    elif standing_condition:
        label = "standing"
    elif sleeping_condition:
        label = "sleeping/lying"
    elif playing_condition:
        label = "playing (arms spread)"
    else:
        label = "unknown/idle"

    # Draw and display
    annotated = image_np.copy()
    mp.solutions.drawing_utils.draw_landmarks(annotated, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    color = (0, 255, 0) if label in ["eating", "running", "playing (arms spread)"] else (0, 0, 255)
    cv2.putText(annotated, label, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
    cv2_imshow(annotated)

    if debug:
        print(f"[{filename}] stride={stride_ratio:.2f}, torso={torso_angle:.1f}, knee={avg_knee_angle:.1f}, hips_y={hips_y:.1f}, shoulders_y={shoulders_y:.1f}")
        print(f" -> bent_knees={bent_knees}, deep_bend={deep_bend}, hips_below={hips_below_shoulders}")
        print(f" -> sitting={sitting_condition}, running={running_condition}, eating={eating_condition}")

    return filename, label

# --- File Upload & Run ---
uploaded = files.upload()
print("\n📁 Processing all selected images...\n")
results_list = []
for filename, data in uploaded.items():
    image = Image.open(io.BytesIO(data)).convert("RGB")
    image_np = np.array(image)[:, :, ::-1]
    results_list.append(classify_activity(image_np, filename, debug=True))

# Save results
with open("activity_results.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Filename", "Detected Activity"])
    writer.writerows(results_list)

print("\n✅ All done! Results saved to 'activity_results.csv'")
